<a href="https://colab.research.google.com/github/0x7o/text2keywords/blob/main/example/keyT5_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Пример обучения на свою задачу

Мы обучим нашу модель keyT5

In [1]:
!pip install transformers sentencepiece

In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import random
from tqdm.auto import tqdm, trange
import os
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('synthetic.csv')

In [6]:
pd.options.display.max_colwidth = 500

In [7]:
df.sample(5)

,category,sentiment,clean_text,synthetic
1126,Общественное питание,Негативная,"Две звезды за кофе! Кофе тут ни при чем! Поняла почему там полумрак! Взяла себе кофе с собой…. Кофе,повторюсь нормальный крепкий Дали мне стаканчик, обмотали салфеткой я еще подумала какие заботливые, что б посетитель не обжегся. Взяла стаканчик с кофе, села в машину…. И из салфетки на меня вылез ТАРАКАН! На этом все!",Сильные стороны:\r\n1. Нормальный крепкий кофе.\r\n2. Заботливое отношение персонала (салфеткой обмотан стаканчик для предотвращения ожогов).\r\n\r\nСлабые стороны:\r\n1. Появление таракана в салоне.
13452,Гостиницы и Размещение,Позитивная,"Отличная гостиница! Вкусное питание. Размещались с детьми 8-9 лет футбольной командой Довольны! В номерах уютно, современно. Жили на 3-м этаже. Есть лифт. Но мы предпочитали ходить по шикарной лестнице. Персонал приветливый и доброжелательный. По пути на завтрак встретили гостя - известного актёра.",Сильные стороны:\r\n1. Вкусное питание.\r\n2. Уютные и современные номера.\r\n3. Наличие лифта.\r\n4. Шикарная лестница.\r\n5. Приветливый и доброжелательный персонал.\r\n6. Встреча с известным актером (может быть воспринято как положительный момент).\r\n\r\nСлабые стороны:\r\n1. Отсутствуют.
9444,Гостиницы и Размещение,Негативная,"НЕ РЕКОМЕНДУЮ ДАННУЮ ГОСТИНИЦУ. Полулюкс, номер 202. Ночь с 23 на 24 мая. Меня разбудил СОТРУДНИК гостиницы в 7:30 утра. Объясняю: напротив номера 202 находится кабинет то ли администрация, то ли фин отдел. Женщина среднего возраста, полная. С открытой дверью слушает музыку, кипятит чайник, громко шумит приборами и дверью, чихает в конце концов! Ничего сверхъестественного она не делала, НО КОМУ КАК НЕ СОТРУДНИКУ ГОСТИНИЦЫ ЗНАТЬ, ЧТО ЛЮДИ ЗДЕСЬ СПЯТ! И ЧТО ШУМЕТЬ, ОСОБЕННО НАПРОТИВ ЗАНЯТОГО Н...","Сильные стороны:\r\n1. отсутствуют\r\n\r\nСлабые стороны:\r\n1. Качество обслуживания (грубость персонала)\r\n2. Уровень шума (шум от сотрудников)\r\n3. Комфортность матраса (жесткость, ощущение пружин)\r\n4. Вода (со вкусом хлорки)"
1049,Общественное питание,Негативная,"Отвратительное место, не рекомендую! Брала сосиску в лаваше, просто, фу, гадость! Выкинула",Сильные стороны:\r\n- отсутствуют\r\n\r\nСлабые стороны:\r\n1. Качество еды
12140,Гостиницы и Размещение,Нейтральная,"Были в этой гостинице двое суток. Гостиница так себе, если это не номера люкс и полу люкс. Все остальные номера на первом этаже в отдельном крыле. И толчок и душевой кабиной находятся в коридоре что крайне не удобно в особенности если ночью заходил в толкан, нужно одеваться, а ещё на каждый шорох выскакивающая бабулька в виде администратора крайне напрягает, при чём выскакивают они всегда с какой нибудь ""пургой"", работают они как я понял сутки через трое разные бабульки в виде уборщиц и откр...","Сильные стороны:\r\n- отсутствуют\r\n\r\nСлабые стороны:\r\n1. Расположение номеров (на первом этаже в отдельном крыле).\r\n2. Удобства (душевые кабины и туалеты) расположены в коридоре.\r\n3. Постоянное присутствие администраторов, вызывающее дискомфорт у гостей.\r\n4. Система открытия двери требует звонка для входа, что неудобно."


Разобьем выборку на обучающую и тестовую, и обучающую превратим в список пар.

In [8]:
df_train, df_test = train_test_split(df.dropna(), test_size=0.5, random_state=1)
pairs = df_train[['clean_text', 'synthetic']].values.tolist()

Инициализируем модель из предобученной

In [9]:
raw_model = '0x7194633/keyt5-large' # или 0x7194633/keyt5-base
model = T5ForConditionalGeneration.from_pretrained(raw_model).cuda();
tokenizer = T5Tokenizer.from_pretrained(raw_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

Обучаем модель! Мы пройдём три эпохи, т.е. покажем данные модели по три раза.

На GPU с размером батча 16 одна эпоха должна занять где-то 15 минут.

In [11]:
batch_size = 2  # сколько примеров показывем модели за один шаг
report_steps = 200  # раз в сколько шагов печатаем результат
epochs = 3  # сколько раз мы покажем данные модели

In [12]:
model.train()
losses = []
for epoch in range(epochs):
    print('EPOCH', epoch)
    random.shuffle(pairs)
    for i in trange(0, int(len(pairs) / batch_size)):
        batch = pairs[i * batch_size: (i + 1) * batch_size]
        x = tokenizer([p[0] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y = tokenizer([p[1] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y.input_ids[y.input_ids == 0] = -100
        loss = model(
            input_ids=x.input_ids,
            attention_mask=x.attention_mask,
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        if i % report_steps == 0:
            print('step', i, 'loss', np.mean(losses[-report_steps:]))

EPOCH 0


  0%|          | 0/3510 [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


step 0 loss 5.48047399520874
step 200 loss 2.3944008055329324
step 400 loss 1.7823199781775474
step 600 loss 1.630821715593338
step 800 loss 1.5073498380184174
step 1000 loss 1.571505212187767
step 1200 loss 1.4348986518383027
step 1400 loss 1.462302526831627
step 1600 loss 1.378653057217598
step 1800 loss 1.319586841762066
step 2000 loss 1.348458801060915
step 2200 loss 1.2704065303504466
step 2400 loss 1.2769295871257782
step 2600 loss 1.2957505254447461
step 2800 loss 1.280878058075905
step 3000 loss 1.2408606101572515
step 3200 loss 1.2007758079469204
step 3400 loss 1.2156839814782143
EPOCH 1


  0%|          | 0/3510 [00:00<?, ?it/s]

step 0 loss 1.2208847391605377
step 200 loss 1.125319589227438
step 400 loss 1.1140150620043279
step 600 loss 1.1072542559355498
step 800 loss 1.0844310918450355
step 1000 loss 1.070392559543252
step 1200 loss 1.106455401480198
step 1400 loss 1.1047616076469422
step 1600 loss 1.0659913511574268
step 1800 loss 1.0865094295144082
step 2000 loss 1.0559314964711666
step 2200 loss 1.0399023917317392
step 2400 loss 1.0222763545811175
step 2600 loss 1.0555642894655466
step 2800 loss 1.0635178932547569
step 3000 loss 1.027028250694275
step 3200 loss 1.0201194141805172
step 3400 loss 1.000020917840302
EPOCH 2


  0%|          | 0/3510 [00:00<?, ?it/s]

step 0 loss 1.0075638837367296
step 200 loss 0.9610247378051281
step 400 loss 0.9778577338159085
step 600 loss 0.9191971064358949
step 800 loss 0.9220712853223085
step 1000 loss 0.9385410468280315
step 1200 loss 0.9100275041162967
step 1400 loss 0.9404223962128162
step 1600 loss 0.9363430947810412
step 1800 loss 0.9480985590815544
step 2000 loss 0.9095564336329699
step 2200 loss 0.9133641919493676
step 2400 loss 0.9161860992759466
step 2600 loss 0.8766468618065119
step 2800 loss 0.8773458527773619
step 3000 loss 0.9069146718084812
step 3200 loss 0.8866751781851053
step 3400 loss 0.8942733799293637


In [13]:
model.eval()

def answer(x, **kwargs):
    inputs = tokenizer(x, return_tensors='pt').to(model.device)
    with torch.no_grad():
        hypotheses = model.generate(**inputs, **kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

Посмотрим, насколько хорошо модель выучила свою тренировочную выборку.

In [14]:
sample = df_train.sample(5)
for i, row in sample.iterrows():
    print(row.clean_text)
    print('real:', row.synthetic)
    print('model: ', answer(row.clean_text))
    print('---')

Коллектив приветлив. Ассортимент товара большой. Кондитерские изделия,колбасы много,молочные продукты и т.д. Очень удобно рядом с домом. График работы очень хороший. Спасибо.
real: Сильные стороны:
1. Приветливый коллектив.
2. Большой ассортимент товаров.
3. Удобное расположение рядом с домом.
4. Хороший график работы.

Слабые стороны:
(отсутствуют)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


model:  Сильные стороны: 1. Приветливый коллектив. 2. Большой ассортимент товаров. 3. Удобное расположение
---
Магазин небольшой, разнообразие товаров. Хотелось бы чтобы товар на витринах оформлялся красиво, и чаще делалась влажная уборка помещения.
real: Сильные стороны:
1. Разнообразие товаров.

Слабые стороны:
1. Оформление товара на витринах.
2. Частота влажной уборки помещения.
model:  Сильные стороны: 1. Разнообразие товаров. Слабые стороны: 1. Оформление товара на
---
Отличная демократичная кофейня Всё супер. Приезжая в Москву на Курский - первым делом - в Кооператив Черный!
real: Сильные стороны:
1. Демократичность.
2. Нет конкретных слабых сторон.

Слабые стороны:
1. Отсутствуют.
model:  Сильные стороны: 1. Демократичность заведения. Слабые стороны: (отсутствуют)
---
По кухне вопросов нет. По обслуживанию очень много вопросов. Сидели довольно долго обставленные грязной посудой и пустыми бокалами, несметрч на то, что нам приносили новые напитки. Приходилось самостоятельно убира

Теперь посмотрим, насколько хорошо модель справляется с ответами на вопросы, которые она не видела.

In [15]:
sample = df_test.sample(5)
for i, row in sample.iterrows():
    print(row.clean_text)
    print('real:', row.synthetic)
    print('model: ', answer(row.clean_text))
    print('---')

Ассортимент богатый, чисто, красиво. Но дороговато по сравнению с другими магазинами. Есть система скидок UDS. Чтобы взвесить самостоятельно овощи и фрукты, надо соображать как это сделать. Не помешало бы инструкцию вывесить рядом. Постоянно требуется персонал. В последнее посещение на кассе работали три парня по виду подростки. Парень еще соображал, как мне отдать сдачу как подошла уборщица и бесцеремонно вмешалась в процесс со своими двумя батончиками
real: Сильные стороны:
1. Богатый ассортимент.
2. Чистота.
3. Красивый интерьер.
4. Наличие системы скидок UDS.

Слабые стороны:
1. Высокие цены по сравнению с другими магазинами.
2. Отсутствие инструкции для самостоятельного взвешивания овощей и фруктов.
3. Недостаток персонала.
4. Некомпетентность сотрудников на кассе.
model:  Сильные стороны: 1. Богатый ассортимент. 2. Чистота. 3. Красивый интерьер.
---
Раньше могла сказать много хорошего, но теперь это просто ужас! Отношение охраны переходит все границы, хамят, переходят на личности

In [17]:
new_model_name = 'keyT5-custom'  # название папки
model.save_pretrained(new_model_name)
tokenizer.save_pretrained(new_model_name)

('keyT5-custom/tokenizer_config.json',
 'keyT5-custom/special_tokens_map.json',
 'keyT5-custom/spiece.model',
 'keyT5-custom/added_tokens.json')